In [1]:
import os
import azureml.core
from azureml.core import Workspace, Experiment, Datastore
from azureml.widgets import RunDetails

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.22.0


In [2]:
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep

print("Pipeline SDK-specific imports completed")

Pipeline SDK-specific imports completed


In [3]:

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

maynard_vscode
maynard_vscode
northeurope
ae0accf0-cf61-4cac-954a-b3979188ad7c


In [4]:
# Default datastore
def_blob_store = ws.get_default_datastore() 
print(def_blob_store)

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-b482fff2-25ca-4c52-b29e-27ca5bb428c4",
  "account_name": "maynardvssaudipxv55vl3xi",
  "protocol": "https",
  "endpoint": "core.windows.net"
}


In [5]:
print("Blobstore's name: {}".format(def_blob_store.name))

Blobstore's name: workspaceblobstore
